In [ ]:
from pynq import Overlay
from pynq import MMIO
from MvpDriver import MvpDriver

In [ ]:
overlay = Overlay("./overlays/MVP_example/MVP_example.bit")
# This shows the documentation for the MVP driver
help(overlay.Module_Validation_Pl_0)

In [ ]:
# Define I/O address and masks
LED_ADDR = 0x00
LED_MASK = 0x0f

BTN_ADDR = 0x00
BTN_MASK = 0x0f

# Define axi BRAM controller address
IP_BASE_ADDRESS       = 0x40000000
AXI_BRAM_CTRL_OFFSET  = 0x00
AXI_BRAM_CTRL_RANGE   = 0x4000

# Make MMIO object for BRAM access directly through AXI
mmio = MMIO(IP_BASE_ADDRESS, AXI_BRAM_CTRL_RANGE)

def read_BRAM_axi(addr:int)->int:
    return mmio.read(AXI_BRAM_CTRL_OFFSET + addr)

def write_BRAM_axi(addr:int, val:int):
    mmio.write(AXI_BRAM_CTRL_OFFSET + addr, val)

In [ ]:
# An external I/O demo
# This section uses real time input
while True:
    # The 4 bit buttons and leds are mapped to address 0x00
    data = overlay.Module_Validation_Pl_0.read_rt_input(LED_ADDR) & LED_MASK
    overlay.Module_Validation_Pl_0.write_oreg(BTN_ADDR, data & BTN_MASK)

In [ ]:
# A demo of the input capture mode
from time import sleep
# before capture the input registers are reset
print(overlay.Module_Validation_Pl_0.read_captured_input(LED_ADDR) & LED_MASK)

# Capture input after 5 seconds
sleep(5)
overlay.Module_Validation_Pl_0.capture()
print(overlay.Module_Validation_Pl_0.read_captured_input(LED_ADDR) & LED_MASK)

# Now the captured input will hold
sleep(5)
print(overlay.Module_Validation_Pl_0.read_captured_input(LED_ADDR) & LED_MASK)

In [ ]:
# BRAM R/W through MVP and AXI MMIO direct validation
for i in range(0, AXI_BRAM_CTRL_RANGE, 4):
    write_BRAM_axi(i, 0xdeadbeef + i)
for i in range(0, AXI_BRAM_CTRL_RANGE, 4):
    data = read_BRAM_axi(i)
    data = overlay.Module_Validation_Pl_0.read_BRAM(i)
    assert(0xdeadbeef + i == data)
    overlay.Module_Validation_Pl_0.write_BRAM(i, 0xdeadbeef - i)
for i in range(0, AXI_BRAM_CTRL_RANGE, 4):
    data = overlay.Module_Validation_Pl_0.read_BRAM(i)
    assert(0xdeadbeef - i == data)

print(f"BRAM r/w success over {AXI_BRAM_CTRL_RANGE} bytes")